In [1]:
import pandas as pd
import tweepy
from IPython.display import clear_output
import time
import pickle

In [2]:
# create connection with Twitter API

CONSUMER_KEY = 'J3uhwWHdmSt3uD69ry8r2kc3B'
CONSUMER_SECRET = 'LeuM7dVfFbXH0bc5fFkZIxlNiUXyjIx4Kcjh58HeWs2TrLijDx'
ACCESS_TOKEN = '327497511-5dNjiGaTQHASljoxNr1qlMOlHrrSB21HBFBDjx2E'
ACCESS_TOKEN_SECRET = 'tHLqwlhHYJWptW3femyq0rbMW6ZItu5yLQ3DFGJqJ8Xlg'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [3]:
ids = []
ids.append(pd.read_csv('data/ff/users.csv')['id'].tolist())
ids = [x for l in ids for x in l]

In [4]:
len(ids)

8825

In [5]:
tweets = []
cont, errors = 0, 0

for user in ids:
    try:
        stuff = api.user_timeline(user_id = user, count = 100, include_rts = True, tweet_mode="extended")
        #tweet.is_quote_status indica se nel tweet c'è il riferimento ad un altro tweet

        for tweet in stuff:
            tweet._json['user_id'] = tweet._json['user']['id']
            if len(tweet._json['entities']['urls']) != 0:
                tweet._json['url'] = tweet._json['entities']['urls'][0]['expanded_url']
            else:
                 tweet._json['url'] = None
            del tweet._json['user'], tweet._json['entities']

            tweets.append(tweet._json)
    except:
        errors += 1
    cont += 1
    clear_output()
    print(str(cont) + ' / 8825   ---   errors: ' + str(errors))
    
    if cont % 500 == 0:
        path = 'porn_tweets_' + str(cont-500) + '-' + str(cont) + '.tw'
        pickle.dump( tweets, open( path , "wb" ) )
        tweets = []
        print ('sleeping...')
        time.sleep(900)

8825 / 8825   ---   errors: 8109


In [2]:
tweets = []

for i in range(17):
    
    path = 'porn_tweets_' + str(i*500) + '-' + str((i+1)*500) + '.tw'
    df = pickle.load( open( path, "rb" ) )
    tweets = tweets + df

In [3]:
df = pd.DataFrame.from_dict(tweets)

In [4]:
# rimuove tutte le interruzioni (spero)

df['full_text'] = df['full_text'].apply(lambda x: x.replace('\r', ' '))
df['full_text'] = df['full_text'].apply(lambda x: x.replace('\n', ' '))
df['full_text'] = df['full_text'].apply(lambda x: x.replace('\t', ' '))

In [5]:
df.to_csv('tweets.csv', index=False, sep='\t', encoding='utf-8-sig')